In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

PROCESSED_DATASET_PATH = "../data/beer-processed.pkl"

In [2]:
df = pd.read_pickle(PROCESSED_DATASET_PATH)

df.head()

,brewery_id,brewery_name,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,review_year,review_month
0,10325,vecchio birraio,1.5,2.0,2.5,stcules,hefeweizen,1.5,1.5,sausa weizen,5.0,47986,2009,2
1,10325,vecchio birraio,3.0,2.5,3.0,stcules,english strong ale,3.0,3.0,red moon,6.2,48213,2009,3
2,10325,vecchio birraio,3.0,2.5,3.0,stcules,foreign / export stout,3.0,3.0,black horse black beer,6.5,48215,2009,3
3,10325,vecchio birraio,3.0,3.0,3.5,stcules,german pilsener,2.5,3.0,sausa pils,5.0,47969,2009,2
4,1075,caldera brewing company,4.0,4.5,4.0,johnmichaelsen,american double / imperial ipa,4.0,4.5,cauldron dipa,7.7,64883,2010,12


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518058 entries, 0 to 1586613
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1518058 non-null  int16  
 1   brewery_name        1518058 non-null  object 
 2   review_overall      1518058 non-null  float16
 3   review_aroma        1518058 non-null  float16
 4   review_appearance   1518058 non-null  float16
 5   review_profilename  1518058 non-null  object 
 6   beer_style          1518058 non-null  object 
 7   review_palate       1518058 non-null  float16
 8   review_taste        1518058 non-null  float16
 9   beer_name           1518058 non-null  object 
 10  beer_abv            1518058 non-null  float64
 11  beer_beerid         1518058 non-null  uint32 
 12  review_year         1518058 non-null  uint16 
 13  review_month        1518058 non-null  uint8  
dtypes: float16(5), float64(1), int16(1), object(4), uint16(1), uint32(

In [4]:
df[["beer_name", "review_profilename", "review_year"]] \
    .groupby(["beer_name", "review_profilename"]) \
    .count() \
    .sort_values(by=["review_year"], ascending=False) \
    .head()

,,review_year
beer_name,review_profilename,
india pale ale,northyorksammy,17
pale ale,buckeyenation,13
zhigulovskoye,globetrotter,12
pale ale,gavage,11
ipa,northyorksammy,11


**Remark:** This means that there were users, who have tried the same beer more than once. This is very important later, as we create a pivot table.

## Queston #1: Which brewery produces the strongest beers by abv?
Note: We have removed entries that report ABV > 20% as unrealistic.

In [5]:
df.get(["brewery_name", "beer_abv"]) \
    .sort_values(by="beer_abv", ascending=False) \
    .drop_duplicates() \
    .head(10) \
    .set_index("brewery_name")

,beer_abv
brewery_name,
the bruery,19.50
sonoran brewing company,19.50
bfm brasserie des franches-montagnes,19.50
sonoran brewing company,19.37
mikkeller aps,19.20
boquébière,19.20
the bruery,18.50
mikkeller aps,18.50
sherbrooke liquor store,18.50


In [6]:
df.get(["brewery_name", "beer_abv"]) \
    .groupby(by="brewery_name") \
    .agg(["mean", "count"]) \
    .sort_values(by=[("beer_abv", "mean")], ascending=False) \
    .head(10)

beer_abv      
                                                         mean count
brewery_name                                                       
shoes brewery                                       15.200000     2
rome brewing company                                13.840000     5
hurlimann brewery                                   13.750000    18
schorschbräu                                        13.366667    27
alt-oberurseler brauhaus                            13.200000     1
rascal creek brewing co.                            13.000000     1
monks porter house                                  12.466667     3
brasserie grain d' orge (brasserie jeanne d'arc...  12.445860   314
tugboat brewing company                             12.187500     8
rinkuki&#371; aluas darykla                        12.000000    11

Apparently, it is the "Shoes Brewery".

In my opinion, the first table only gives us the name of a brewery that has produced a single, most alcoholic beer. However, I understand the quesion as a case to find a brewery that produces strong beers on average.
From the second list, we can see it is the "Shoes Brewery".
Still, I would argue that it could just as well be "Hurlimann", "Schorschriau" or "Brasserie Grain d'Orge", because their average is "statistically stronger".

## Question #2. If you had to pick 3 beers to recommend to someone, how would you approach the problem?

The best approach would be to create a recommender system aka _collaborative filtering_.
Unfortunately, we know very little of the users (in terms of vectors), apart from how they graded the beers - we So it a bit hard to start.
In addition, if we have a new user, we don't know anything about his/her preference, which adds to the difficulty (it's not impossible, but just challenging).

However, what we can do is to propose a three step problem.
First, we ask the newcomer what beers does he/she likes (e.g. in terms of let's say brewery, alcohol or style).
This will give us some information for search.
Then, we can ask another question: "what is the most important for you?". The answer to this question will prompt us to pick the right review metric (e.g. appearence, palate, or overall as the default).
Finally, we can use the existing correlations to find the two remaining beers that will be the most similar to the first one using the Pearson correlation coefficient, kind of making a "cheap" recommender system.
It basically looks at the problem as "similar users should grade similar beers... similarly".

One more thing before we start.
We choose to pick users that scored at least 1000 beers.
We also choose beers that were given at least 1000 scores.
Why?
* Because single opinions don't mean much statistically... we may have a beer that scores 5/5, but if only one person drank it, it doesn't tell much.
* Because I will be pivoting the table, and it won't fit into the memory. (I can convert it to a sparse representation, but that's a bit beyond.)

In [7]:
profilenames = df["review_profilename"].value_counts() \
    .to_frame() \
    .rename(columns={"review_profilename": "counts"}) \
    .sort_values(by="counts", ascending=False) \
    .query("counts > 1000") \
    .reset_index() \
    .get("index") \
    .tolist()

beernames = df["beer_name"].value_counts() \
    .to_frame() \
    .rename(columns={"beer_name": "counts"}) \
    .sort_values(by="counts", ascending=False) \
    .query("counts > 1000") \
    .reset_index() \
    .get("index") \
    .tolist()

print(f"#Profiles: {len(profilenames)}, #beers: {len(beernames)}")

#Profiles: 223, #beers: 209


In [8]:
review_columns = [
    'review_overall',
    'review_aroma',
    'review_appearance',
    'review_palate',
    'review_taste',
]

reviews = df \
    .query("review_profilename in @profilenames and beer_name in @beernames") \
    .get(["review_profilename", "beer_name"] + review_columns)

profile_index_encoder = LabelEncoder().fit(reviews["review_profilename"])
beer_index_encoder = LabelEncoder().fit(reviews["beer_name"])

reviews["profile_index"] = profile_index_encoder.transform(reviews["review_profilename"])
reviews["beer_index"] = beer_index_encoder.transform(reviews["beer_name"])

# because we have duplicated entries in terms of (profile, beer)-pairs across time:
reviews = reviews.groupby(by=["review_profilename", "beer_name"]).mean().reset_index()

reviews.sample(3)

,review_profilename,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,profile_index,beer_index
27031,scruffwhor,spaten optimator,5.0,2.5,3.0,5.0,3.5,179,176
15340,huhzubendah,stone ipa (india pale ale),4.0,4.0,4.5,4.0,4.0,101,181
18902,lacqueredmouse,hop rod rye,4.5,5.0,5.0,4.0,4.5,125,86


In [9]:
# side note: this cell basically pivots the table... unfortunately, I had no luck with neither .pivot
# nor .unstack methods. That was due to me forgetting the "fime-dimension" (same people drinking the same beer)
# I fixed it earlier, but chose to leave this code.

review_metric = "review_overall"

def prepare_pivot_table(reviews, review_metric):
    pidx = reviews["profile_index"].sort_values().unique()
    bidx = reviews["beer_index"].sort_values().unique()

    R = reviews.get(["profile_index", "beer_index", review_metric])

    X = pd.DataFrame(
            np.array([np.tile(bidx, len(pidx)), np.repeat(pidx, len(bidx))]).T,
            columns=["beer_index", "profile_index"],
        ).merge(R, on=["beer_index", "profile_index"], how="left")

    Y = []
    for p, item in X.groupby("profile_index"):
        entry = pd.Series(item[review_metric], name=p).reset_index(drop=True)
        Y.append(entry)

    return pd.concat(Y, axis=1)

In [10]:
# prepare_pivot_table(reviews, review_metric)

In [11]:
def get_more_beers(similar_to, review_metric="review_overall", n_beers=2, debug=False):
    if similar_to not in reviews["beer_name"].values:
        raise KeyError("We don't have enough data, try something else.")

    beer_index = beer_index_encoder.transform([similar_to])[0]

    Y = prepare_pivot_table(reviews, review_metric).T
    correlations = Y.corrwith(Y[beer_index]).sort_values(ascending=False)
    if debug:
        print(correlations.iloc[:n_beers + 1])
    return beer_index_encoder.inverse_transform(correlations.index[1:n_beers + 1]).tolist()

In [12]:
get_more_beers("#9", n_beers=5)

['bass pale ale',
 'punkin ale',
 'duvel',
 'hercules double ipa',
 'samuel adams white ale']

In [13]:
get_more_beers("duvel", n_beers=5)

["samuel smith's, the famous taddy porter",
 'sierra nevada stout',
 'anchor porter',
 'ommegang (abbey ale)',
 'punkin ale']

In [14]:
get_more_beers("christmas ale", review_metric="review_aroma", n_beers=5)

['nut brown ale',
 'allagash white',
 'left hand milk stout',
 'hopdevil ale',
 'founders imperial stout']

In [15]:
get_more_beers("christmas ale", review_metric="review_appearance", n_beers=5)

['1554 enlightened black ale',
 'saison dupont',
 'weihenstephaner hefeweissbier',
 'westmalle trappist dubbel',
 'yuengling traditional lager']

In [16]:
# pick a few
# beer_index_encoder.inverse_transform(np.arange(100))

## Question #3. What are the factors that impacts the quality of beer the most?
Here, we assume the ultimate beer quality indicator is `review_overall` (it's probaby a linear combination of the other scores, but not the exact simple mean).
Apart from these scores, we also have the ABV figure and two time variables derived from the timestamp (here we assume that only the year and month may have an impact, while days, hours, minues are noise). After all, we would suspect that people do may have preference for certain tastes during e.g. winter time.

So now, we will take these variables and see how they correlate to `review_overall`.

In [17]:
quality_df = df[review_columns + ["beer_style", "beer_abv"] + ["review_year", "review_month"]].copy()

In [18]:
quality_df.corr().get("review_overall") \
    .to_frame() \
    .rename(columns={"review_overall": "correlation"}) \
    .sort_values(by="correlation", ascending=False)

,correlation
review_overall,1.000000
review_taste,0.787249
review_palate,0.699056
review_aroma,0.612891
review_appearance,0.498571
beer_abv,0.141857
review_year,0.026372
review_month,-0.011325


As we can see, the most impactful figure (or at least the most highly correlated figure) is the `taste` (not surpising?). Interestingly, it also turns out that the ABV has a negligible impact on the review score.
Similarly, the "seasonal" figures seem to be very weakly correlated, practically not correlated at all.

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X = quality_df[
    [c for c in review_columns if not c.endswith("overall")] + ["beer_abv", "review_year", "review_month"]
]

y = quality_df["review_overall"]

pipe = make_pipeline(StandardScaler(), LinearRegression())
pipe.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [20]:
pd.DataFrame({k:v for k, v in zip(X.columns, pipe.steps[1][1].coef_)}, index=["linear_coeffs"]) \
    .transpose() \
    .abs() \
    .sort_values("linear_coeffs", ascending=False)

,linear_coeffs
review_taste,0.403835
review_palate,0.184140
beer_abv,0.097895
review_aroma,0.053997
review_appearance,0.029624
review_month,0.003568
review_year,0.002198


This is basically a confirmation of the above statement. While we haven't confirmed that the linear model is indeed the best (no test and control groups), we have a strong reason to believe so.
This table shows the relative "strengh" of the components in the linear combination.
Note that the table isn't exactly reflecting the correlation, but this is due to applying the feature scaling.

## Question #4. I enjoy a beer which aroma and appearance matches the beer style. What beer should I buy?
Here, the approach is to group the previously created `quality_df` table by `beer_style` variable and see which of the group is associated with the largest `appearance` and `aroma` mean.
We will also look at the standard deviation, since there may be large discrepencies between reviews for
some beers with extreme mean values.

In [21]:
selected_df = quality_df.groupby(by="beer_style").agg(["mean", "std"]) \
    .sort_values(by=[
        ("review_aroma", "mean"),
        ("review_appearance", "mean"),
    ], ascending=False) \
    .reset_index() \
    .get(["beer_style", "review_aroma", "review_appearance"])

selected_df

beer_style review_aroma            \
                                              mean       std   
0    american double / imperial stout     4.164062  0.569282   
1                             eisbock     4.156250  0.529080   
2                    quadrupel (quad)     4.132812  0.544025   
3                   american wild ale     4.132812  0.561834   
4                  lambic - unblended     4.125000  0.534554   
..                                ...          ...       ...   
99                           happoshu     2.597656  0.761265   
100            american adjunct lager     2.478516  0.715567   
101                  low alcohol beer     2.437500  0.849931   
102              american malt liquor     2.410156  0.850248   
103                       light lager     2.144531  0.779988   

    review_appearance            
                 mean       std  
0            4.164062  0.515265  
1            3.962891  0.495995  
2            4.121094  0.513658  
3            4.011719  0.503281  
4            3.917969  0.519880  
..                ...       ...  
99           2.919922  0.797179  
100          2.785156  0.735151  
101          2.896484  0.854466  
102          2.835938  0.809876  
103          2.406250  0.841018  

[104 rows x 5 columns]

Here is the list. The "american double / imperial stout", followed by others. The standard deviation for those "winning" beers is also lower, indicating a more coherent judgement.